<a href="https://colab.research.google.com/github/joexu22/clifford/blob/master/GPTNeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Me Setting Up GPTNeo

[GPTNeo](https://github.com/EleutherAI/GPTNeo) by [EleutherAI](eleuther.ai)

[Eleuther's Discord](https://discord.gg/BK2v3EJ) 

Setup Instructions

1. You need to turn on TPU Runtime
  - Go to "Runtime" -> Choose "Change runtime type" and select "TPU" under "hardware accelerator".

In [1]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt

Cloning into 'GPTNeo'...
remote: Enumerating objects: 3747, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 3747 (delta 77), reused 121 (delta 69), pack-reused 3609
Receiving objects: 100% (3747/3747), 1.44 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (2167/2167), done.
/content/GPTNeo
     |████████████████████████████████| 368kB 6.7MB/s 
     |████████████████████████████████| 14.4MB 18.8MB/s 
     |████████████████████████████████| 112kB 51.4MB/s 
     |████████████████████████████████| 394.7MB 41kB/s 
     |████████████████████████████████| 3.4MB 39.5MB/s 
     |████████████████████████████████| 2.9MB 44.6MB/s 
     |████████████████████████████████| 1.5MB 47.4MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 184kB 53.9MB/s 
     |████████████████████████████████| 2.2MB 37.8MB/s 
     |████████████████████████████████| 1.0MB 45.6MB/s 
     |█████████████████████

In [2]:
#@title Sampling Only Dataset

dataset = 'Sampling_Only'

In [3]:
#@title Use Pretrained Model
pretrained_model = None

Notes -

Using TPUs requires Cloud Filesystems

Using Google Cloud: https://console.cloud.google.com/

Making Google Cloud Bucket: https://console.cloud.google.com/storage

In [4]:
#@title Connect To Google Cloud Bucket - Step 1 (Authentication)
from google.colab import auth
auth.authenticate_user()
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: kiro66500@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] kiro66500@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [kiro66500@gmail.com].

Pick cloud project to use: 
 [1] databasetest-

In [5]:
#@title Use Cloud Bucket - Step 2 (Locate/Use Bucket Location)
path_to_cloud_bucket = 'gs://my-machine-learning-bucket' #@param {type:"string"}

In [6]:
#@title Use Dataset (Currently: Sampling Only)
import os
dataset = "Sampling_Only"
pass

In [7]:
#@title Sampling Only Configuration - Step 1 (CD to GPTNeo Directory)
# strange how they don't let me use %cd and %%writefile together
%cd /content/GPTNeo

/content/GPTNeo


In [8]:
#@title Sampling Only Configuration - Step 2 (Write File)
%%writefile configs/dataset_configs/Sampling_Only.json

{
  "path": "gs://my-machine-learning-bucket/datasets/Sampling_Only/Sampling_Only*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}


Writing configs/dataset_configs/Sampling_Only.json


In [8]:
#@title Sampling Only Configuration - Step 3 (Write Model Config)
%%writefile configs/GPT3_2-7B.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 600000,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 4,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["pile", null, null, null]],
    "model": "GPT",
    "model_path": "gs://my-machine-learning-bucket/GPT3_2-7B",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:n_vocab,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16"
}

Writing configs/GPT3_2-7B.json


In [9]:
# @title Get Pretrained Models - Step 1 (Uncomment If Missing Weights)
pretrained_model = 'GPT3_2-7B' #@param ["GPT3_XL", "GPT3_2-7B"]
# !wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/$pretrained_model/"
path_to_local_weights = f"/content/GPTNeo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"

# URL = f"http://eaidata.bmk.sh/data/gptneo-release/{pretrained_model}/"
# FOLDER_NAME = "GPT3_XL"
# !curl $URL | grep -i "</a>" | sed -n 's/.*href="\([^"]*\).*/\1/p' | sed "s|^|$URL|" | xargs -n 1 -P 4 wget -P $pretrained_model
# path_to_local_weights = pretrained_model

In [10]:
# @title Get Pretrained Models - Step 2 (Uncomment If Missing Weights)

# upload to your bucket
bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]
# !gsutil -m cp -r $path_to_local_weights $bucket_base

In [11]:
# @title Confirm Bucket
!gsutil ls $bucket_base

gs://my-machine-learning-bucket/GPT3_2-7B/


In [12]:
# @title Get The GPT3_2-7B Config
%cd /content/GPTNeo
!wget "https://the-eye.eu/public/AI/gptneo-release/GPT3_2-7B/config.json" -P configs/local_configs/

/content/GPTNeo
--2021-05-10 05:33:10--  https://the-eye.eu/public/AI/gptneo-release/GPT3_2-7B/config.json
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 937 [application/json]
Saving to: ‘configs/local_configs/config.json’

config.json         100%[===================>]     937  --.-KB/s    in 0s      

2021-05-10 05:33:10 (133 MB/s) - ‘configs/local_configs/config.json’ saved [937/937]



In [13]:
# @title Modify config for colab. 
  
import json
from pprint import pprint

path_to_model = "" #@param {type:"string"}
batch_size = 8 #@param {type:"integer"}
dset = "pile"  #@param {type:"string"}
mesh_shape = "x:4,y:2" #@param {type:"string"}
train_steps = 1000 #@param {type:"integer"}
steps_per_checkpoint = 500 #@param {type:"integer"}
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'/content/GPTNeo/configs/local_configs/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: gs://my-machine-learning-bucket/GPT3_2-7B

{'activation_function': 'gelu',
 'ada_epsilon1': '1e-30',
 'ada_epsilon2': 0.001,
 'attention_types': [[['global', 'local'], 16]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,embd:y',
 'lr': 0.00016,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:64,y:4',
 'model_path': 'gs://neo-d/models/GPT3_2-7B',
 'n_ctx': 2048,
 'n_embd': 2560,
 'n_head': 20,
 'n_layer': 32,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'predict_batch_size': 1,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'ac

In [14]:
# @title Sample Text
%%writefile example_prompt.txt

So, GPTNeo. I have set you up.
What are your further instructions?

List Them Here:


Writing example_prompt.txt


In [15]:
# @title Quick Train
# !python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

In [16]:
# @title Run Code
!python3 main.py --model $pretrained_model --steps_per_checkpoint 500 --tpu colab --predict --prompt example_prompt.txt

2021-05-10 05:33:30.096122: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 400000
Downloading: 100% 1.04M/1.04M [00:00<00:00, 5.94MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.27MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 7.36MB/s]
Saving config to gs://my-machine-learning-bucket/GPT3_2-7B
2021-05-10 05:33:40.025131: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-10 05:33:40.027351: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-10 05:33:40.039784: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-10 05:33:40.039854: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kern